<a href="https://colab.research.google.com/github/MengOonLee/WebScrapy/blob/master/Groceries/Lotus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install --no-cache-dir -qU scrapy selenium

In [4]:
from selenium import webdriver
import scrapy
import time

options = webdriver.chrome.options.Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--enable-javascript")
options.add_argument("--enable-cookies")
options.add_argument("--disable-notifications")
options.add_argument("--disable-web-security")
options.add_argument("--incognito")
driver = webdriver.Chrome(options=options)

url = "https://www.lotuss.com.my/en/category/grocery"
driver.get(url)
time.sleep(10)
# selector = scrapy.Selector(text=driver.page_source.encode("utf-8"))
selector = scrapy.Selector(text=driver.page_source)
products = selector.css("div.product-grid-item a::attr(href)").getall()
driver.close()

products

['/en/product/mamee-premium-mi-tarik-vegetarian-5x80g-1188429',
 '/en/product/jati-100-beras-wangi-aaa-10kg-4109589',
 '/en/product/mamee-premium-mi-tarik-chicken-5x79g-3641368',
 '/en/product/mamee-premium-mi-tarik-curry-5x80g-1188372',
 '/en/product/mamee-premium-mi-tarik-vege-curry-5x78g-74744348',
 '/en/product/mamee-premium-mi-tarik-duck-5x81g-376221',
 '/en/product/74913077',
 '/en/product/74889192',
 '/en/product/74889206',
 '/en/product/kelloggs-coco-pops-400g-199125',
 '/en/product/nestle-honey-stars-econopack-500g-125946',
 '/en/product/rasaku-coconut-milk-200mlx6-72240474',
 '/en/product/skippy-p-b-chocolate-stripes-530g-71495916',
 '/en/product/cadbury-dairy-milk-hazelnut-160g-71667245',
 '/en/product/naxnax-choco-marie-biscuit-300g-74704664']

In [11]:
%%writefile Lotus.py
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions
import scrapy
from scrapy.crawler import CrawlerProcess

class LotusSpider(scrapy.Spider):
    name = 'Lotus'

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        options = webdriver.chrome.options.Options()
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--enable-javascript")
        options.add_argument("--enable-cookies")
        options.add_argument("--disable-notifications")
        options.add_argument("--disable-web-security")
        options.add_argument("--incognito")
        self.driver = webdriver.Chrome(options=options)

    def start_requests(self):
        urls = [
            "https://www.lotuss.com.my/en/category/grocery"
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        self.driver.get(response.url)

        try:
            WebDriverWait(self.driver, 10).until(
                expected_conditions.presence_of_element_located(
                    (By.CLASS_NAME, "product-grid-item"))
            )

            selector = scrapy.Selector(text=self.driver.page_source)

            product_pages = selector.css(
                "div.product-grid-item a::attr(href)"
            ).getall()

            print(product_pages)
        finally:
            self.driver.quit()

process = CrawlerProcess()
process.crawl(LotusSpider)
process.start()

Overwriting Lotus.py


In [ ]:
%%bash
python Lotus.py